<a href="https://colab.research.google.com/github/AlexeyTri/MyMachineLearningSeminars/blob/main/%D0%9C%D0%BE%D0%B4%D0%B5%D0%BB%D0%B8_%D0%B2%D0%BD%D0%B8%D0%BC%D0%B0%D0%BD%D0%B8%D1%8F_%D0%B8_%D1%82%D1%80%D0%B0%D0%BD%D1%81%D1%84%D0%BE%D1%80%D0%BC%D0%B5%D1%80%D1%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Attention is all you need. Трансформеры.


#Бибилиотеки, загрузка данных, код обучения

*http://opus.nlpl.eu/Wikipedia-v1.0.php - выборка Wikipedia en-ru  и ru-en*

*данные раземещены на GoogleDrive*


In [1]:
!pip install --quiet -U transformers torch sentencepiece

In [2]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import gridspec

from tqdm.notebook import tqdm
import numpy as np
import  pandas as pd
import seaborn as sns
import torch
import scipy
# AutoModel это класс создающий зкземпляр соответствующей модели, с переданными на вход настроенными весами, конфигурацией и словарем
from transformers import AutoModel, AutoTokenizer
# конфигурация модели Marian https://huggingface.co/docs/transformers/v4.25.1/en/model_doc/marian#transformers.MarianConfig
# This model is also a PyTorch torch.nn.Module subclass. Use it as a regular PyTorch
from transformers import MarianMTModel, MarianTokenizer

from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device


'cpu'

In [5]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [6]:
with open('/gdrive/MyDrive/Wikipedia.en-ru.en') as f:
    en_all_texts = f.read().splitlines()
with open('/gdrive/MyDrive/Wikipedia.en-ru.ru') as f:
    ru_all_texts = f.read().splitlines()

In [7]:
ru_texts = []
en_texts = []

for ru_text, en_text in zip(ru_all_texts, en_all_texts):
    if len(ru_text) < 100 and len(en_text) < 100:
        ru_texts.append(ru_text.lower())
        en_texts.append(en_text.lower())

In [8]:
len(ru_texts), len(en_texts)

(306887, 306887)

In [9]:
ru_texts[:20]

['через неделю, 1 сентября, германия начала вторжение в польшу.',
 '29 марта 2004 года литва вступила в нато.',
 'число занятых в промышленности — 27,5 % трудоспособного населения.',
 'традиционно в россии популярны настольные интеллектуальные игры.',
 'о статусе соционики существуют противоположные мнения.',
 'д.',
 '2004.',
 '2014.',
 '3-16.',
 '2014.',
 '24-40.',
 'гос.',
 'церковь строилась в качестве кафедрального собора неподалёку от княжеского терема.',
 'г.',
 '* всемирная история.',
 'началось стачечное движение и забастовки на железных дорогах россии.',
 'на кавказе начались столкновения армян с азербайджанцами, продолжавшиеся в 1905—1906 годах.',
 'жителей (1999), с пригородами — 87 800 жителей.',
 'д.).',
 'n, (2006).']

Блок кода для обучения

In [10]:
def train_on_batch(model, x_batch, y_batch, optimizer, loss_function):
    model.train()
    optimizer.zero_grad()

    output = model(x_batch.to(model.device), y_batch.to(model.device))

    loss = loss_function(output.transpose(1,2),
                         y_batch.to(model.device))
    
    loss.backward()
    optimizer.step()
    return loss.cpu().item()

TODO => CALLBACK

In [11]:
def train_epoch(train_generator, model, loss_function, optimizer, callback=None):
    epoch_loss = 0
    total = 0
    for it, (batch_of_x, batch_of_y) in enumerate(train_generator):
        batch_loss = train_on_batch(model, batch_of_x, batch_of_y, optimizer, loss_function)

        if callback is not None:
            with torch.no_grad():
                callback(model, batch_loss)
        epoch_loss += batch_loss*len(batch_of_x)
        total += len(batch_of_x)

    return epoch_loss/total

In [12]:
def trainer(count_of_epoch,
            batch_size,
            dataset,
            model,
            loss_function,
            optimizer,
            lr=0.001,
            callback=None):
    optima = optimizer(model.parameters(), lr=lr)

    iterations = tqdm(range(count_of_epoch), desc='epoch')
    iterations.set_postfix({'train epoch loss': np.nan})
    for it in iterations:
        batch_generator = tqdm(
            torch.utils.data.DataLoader(dataset=dataset,
                                        batch_size=batch_size,
                                        shuffle=True, pin_memory=True),
                               leave=False, total=len(dataset)//batch_size+(len(dataset)%batch_size>0))
        epoch_loss = train_epoch(train_generator=batch_generator,
                                 model=model,
                                 loss_function=loss_function,
                                 optimizer=optima,
                                 callback=callback)
        iterations.set_postfix({'train epoch loss': epoch_loss})

# Attn RNN


\begin{align}
        \mathbf{с} &= Attn(q,K,V)=\sum_{i=1} v_i SoftMax_i a(k_i,q)
    \end{align}

    * q - вектор-запрос, для которого хотим вычислить контекст
    * K = (к_1,...,k_n) - вектор-ключи, сравниваемые с запросом
    * V = (v_1,...,v_n) - векторы-значения, образующие контекст
    * a(k_i,q) - оценка релевантности (сходства) ключа k_i запросу q
    * c - искомый вектор контекста 

In [14]:
class Encoder(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device

    def __init__(self, vocab_size, emb_dim=30, hidden_dim=30):
        super(Encoder, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, emb_dim)
        self.lstm = torch.nn.LSTM(emb_dim, hidden_dim, batch_first=True)

    def forward(self, input):
        r'''
        :param input: тензор размера batch_size x seq_len --- список токенов
        '''

        act = self.embedding(input)
        act, hidden = self.lstm(act)
        return act, hidden


In [15]:
class Decoder(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device

    def __init__(self, vocab_size, emb_dim=30, hidden_dim=30):
        super(Decoder, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, emb_dim)
        self.attention = torch.nn.MultiheadAttention(emb_dim, 1)
        self.lstm = torch.nn.LSTM(emb_dim, hidden_dim, batch_first=True)
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, input, encoder_outputs, hidden):
        r"""
        :param input: тензор размера batch_size x seq_len
        """
        act = self.embedding(input)
        act, _ = self.attention(act.transpose(0,1),
                                encoder_outputs.transpose(0,1),
                                encoder_outputs.transpose(0,1))
        act = act.transpose(0,1)
        act, hidden = self.lstm(act, hidden)
        return self.linear(act), hidden

In [16]:
class seq2seq(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device
    
    def __init__(self, vocab_size, emb_dim=30, hidden_dim=30):
        super(seq2seq, self).__init__()
        self.vocab_size = vocab_size
        self.encoder = Encoder(vocab_size, emb_dim, hidden_dim)
        self.decoder = Decoder(vocab_size, emb_dim, hidden_dim)

    def forward(self, input, decoder_input=None, max_seq_len=64):
        r'''
        '''
        encoder_output, hidden = self.encoder(input)

        if decoder_input is None:
            translated_scores = torch.zeros(len(input),
                                            max_seq_len,
                                            self.vocab_size).to(self.device)
            translated_scores[:, 0, input[:,0]] = 1.
            for i in range(1, max_seq_len):
                translated_scores[:, i:i+1], hidden = self.decoder(
                    torch.argmax(translated_scores[:, i-1:i], axis=-1),
                    encoder_output,hidden)
        else:
            translated_scores, _ = self.decoder(decoder_input, encoder_output, hidden)
        return translated_scores

In [17]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-uncased', verbose=False)
tokenizer.vocab_size

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.72M [00:00<?, ?B/s]

105879

In [18]:
en_text_tenzors = tokenizer(en_texts[:20000],
                            return_tensors='pt', max_length=64, padding=True)
ru_text_tenzors = tokenizer(ru_texts[:20000],
                            return_tensors='pt', max_length=64, padding=True)

In [19]:
dataset_train_pt = torch.utils.data.TensorDataset(
    en_text_tenzors['input_ids'][:19000], ru_text_tenzors['input_ids'][:19000])
dataset_test_pt = torch.utils.data.TensorDataset(
    en_text_tenzors['input_ids'][19000:], ru_text_tenzors['input_ids'][19000:])

In [ ]:
model = seq2seq(tokenizer.vocab_size)
model.to(device)

In [21]:
# https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
loss_function = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam

In [ ]:
trainer(count_of_epoch=15, 
        batch_size=64, 
        dataset=dataset_train_pt,
        model=model, 
        loss_function=loss_function,
        optimizer = optimizer,
        lr=0.001,
        callback=None)

epoch:   0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/297 [00:00<?, ?it/s]

  0%|          | 0/297 [00:00<?, ?it/s]

In [ ]:
tokens_en = tokenizer(['in 1924, most of the site was converted to a museum.'], return_tensors='pt')
tokens_ru = tokenizer(['в 1924 году крепость стала музеем'],return_tensors='pt')
with torch.no_grad():
    answer = torch.argmax(
        model(tokens_en['input_ids'].to(model.device),
              tokens_ru['input_ids'].to(model.device), max_seq_len=10),axis=-1)
tokenizer.decode(answer[0])

In [ ]:
with torch.no_grad():
    encoder_outputs, _=model.encoder(
        tokens_en['input_ids'].to(model.device))
    decoder_embedding = model.decoder.embedding(
        tokens_ru['input_ids'].to(model.device))
    
    act, act_weight = model.decoder.attention(
        decoder_embedding.transpose(0,1),
        encoder_outputs.transpose(0,1),
        encoder_outputs.transpose(0,1))

In [ ]:
attention_frame = pd.DataFrame(
    act_weight[0].cpu().numpy(),
    index=tokenizer.convert_ids_to_tokens(
        tokens_ru['input_ids'][0].cpu().numpy()),
    columns=tokenizer.convert_ids_to_tokens(
        tokens_en['input_ids'][0].cpu().numpy()))

In [ ]:
sns.heatmap(data=attention_frame)

plt.show()

# ТРАНСФОРМЕР

**seq2seq** -  RNN для снтеза последовательностей

In [ ]:
class Encoder(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device

    def __init__(self, vocab_size, emb_dim=30, hidden_dim=30):
        super(Encoder, self).__init__()
        self.layers = torch.nn.Sequential()
        self.embedding = torch.nn.Embedding(vocab_size, emb_dim)
        self.attention1 = torch.nn.MultiheadAttention(emb_dim, 1)
        self.linear1 = torch.nn.Linear(emb_dim, hidden_dim)
        self.relu = torch.nn.ReLU()
        self.attention2 = torch.nn.MultiheadAttention(hidden_dim,1)
        self.linear2 = torch.nn.Linear(hidden_dim, hidden_dim)

    def forward(self, input):
        r'''
        :param input: тензор размера batch_size x seq_len --- список токенов
        '''
        input = input.transponse(0,1)
        act = self.embedding(input)
        act, _ = self.attention1(act, act, act)
        act = self.relu(act)
        act = self.linear1(act)
        act = self.relu(act)
        act, _= self.attention2(act, act, act)
        act = self.relu(act)
        act = self.linear2(act)
        return act.transpose(0, 1)

In [ ]:
class Decoder(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device

    def __init__(self, vocab_size, emb_dim=30, hidden_dim=30):
        super(Decoder, self).__init__()
        self.embedding = torch.nn.Embedding(vocab_size, emb_dim)
        self.attention = torch.nn.MultiheadAttention(emb_dim, 1)
        self.linear1 = torch.nn.Linear(emb_dim, hidden_dim)
        self.relu = torch.nn.ReLU()

        self.attention2 = torch.nn.MultiheadAttention(hidden_dim, 1)
        self.linear2 = torch.nn.Linear(hidden_dim, hidden_dim)

        self.linear = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, input, encoder_outputs):
        r'''
        :param input: тезор размера batch_size x seq_len
        '''
        input = input.transpose(0, 1)
        act = self.embedding(input)

        act, _ = self.attention(act, 
                                encoder_outputs.transpose(0, 1), 
                                encoder_outputs.transpose(0, 1))
        
        act = self.relu(act)
        act = self.linear1(act)
        act = self.relu(act)
        act, _ = self.attention2(act, 
                                 encoder_outputs.transpose(0, 1), 
                                 encoder_outputs.transpose(0, 1))
        act = self.relu(act)
        act = self.linear2(act)

        return self.linear(act).transpose(0, 1)


In [ ]:
class seq2seq(torch.nn.Module):
    @property
    def device(self):
        return next(self.parameters()).device
    def __init__(self, vocab_size, emb_dim=30, hidden_dim=30):
        super(seq2seq, self).__init__()
        self.vocab_size = vocab_size
        self.encoder = Encoder(vocab_size, emb_dim, hidden_dim)
        self.decoder = Decoder(vocab_size, emb_dim, hidden_dim)

    def forward(self, input, decoder_input=None, max_seq_len=64):
        r'''
        '''
        encoder_output = self.encoder(input)
        if decoder_input is None:
            translated_scores = torch.zeros(len(input),
                                            max_seq_len,
                                            self.vocab_size).to(self.device)
            translated_scores[:,0,input[:,0]] = 1
            for i in range(1, max_seq_len):
                translated_scores[:, i:i+1] = self.decoder(
                    torch.argmax(translated_scores, axis=-1),
                    encoder_output)[:, i:i+1]
        else:
            translated_scores = self.decoder(
                decoder_input, encoder_output)
        return translated_scores

In [ ]:
model = seq2seq(tokenizer.vocab_size)
model.to(device)

In [ ]:
tokens_en = tokenizer(['in 1924, most of the site was converted to a museum.'],return_tensors='pt')
tokens_ru = tokenizer(['в 1924 году крепость стала музеем.'],return_tensors='pt')
with torch.no_grad():
    answer = torch.argmax(
        model(tokens_en['input_ids'].to(model.device),
              tokens_ru['input_ids'].to(model.device)),
              axis=-1
    )
tokenizer.decode(answer[0])

In [ ]:
loss_function = torch.nn.CrossEntropyLoss(ignore_index=0)
optimizer = torch.optim.Adam

In [ ]:
trainer(count_of_epoch=5,
        batch_size=64,
        dataset=dataset_train_pt,
        model=model,
        loss_function=loss_function,
        optimizer=optimizer,
        lr=0.001,
        callback=None)

In [ ]:
tokens_en = tokenizer(['in 1924, most of the site was converted to a museum.'], return_tensors='pt')
tokens_ru = tokenizer(['в 1924 году крепость стала музеем.'], return_tensors='pt')
with torch.no_grad():
    answer = torch.argmax(
        model(tokens_en['input_ids'].to(model.device), 
              tokens_ru['input_ids'].to(model.device)), 
        axis=-1)
tokenizer.decode(answer[0])

# **Tensor2Tensor**

In [ ]:
from transformers import MarianMTModel
tokenizer = MarianTokenizer.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
model = MarianMTModel.from_pretrained('Helsinki-NLP/opus-mt-ru-en')
model.to(device)

In [ ]:
batch_x = tokenizer.prepare_seq2seq_batch(src_texts=['Привет мир, меня зовут Трансформер'],return_tensors='pt')

In [ ]:
tokenizer.batch_decode(model.generate(**batch_x.to(device)))

# Токенизация Byte Pair Encoding **BPE**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE', verbose=False)
tokenizer.vocab_size

In [ ]:
tokenizer.tokenize('Hello Mathematic, my Name is Alexey, how are you&')

# Bidirectional Encoder Representations from Transfomers - BERT

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', verbose=False)
model = AutoModel.from_pretrained('bert-base-multilingual-cased')
model.to(device)

In [ ]:
tokens = tokenizer(['Hello World','Alexey Litovchenko'], return_tensors='pt', padding=True)
with torch.no_grad():
    output =  model(**tokens.to(device))

In [ ]:
tokens['input_ids']

In [ ]:
output[0].shape, output[1].shape

In [ ]:
tokens

#Language-agnostic BERT Sentence Embedding **BERT**

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/LaBSE', 
                                          verbose=False)
model = AutoModel.from_pretrained('sentence-transformers/LaBSE')
model.to(device)


In [ ]:
tokens = tokenizer(['Moscow Institute of Physics and Technology', 
                    'Московский Физико-Технический Институт', 
                    'Московский Государственный Университет', 
                    'Moscow State University'], 
                   return_tensors='pt', padding=True)

with torch.no_grad():
    output = model(**tokens.to(device))

sentence_embeding = output[1].cpu().numpy()

In [ ]:
sentence_embeding.shape

In [ ]:
frame = pd.DataFrame(np.round(
    scipy.spatial.distance.cdist(sentence_embeding, sentence_embeding, 
                                 metric='cosine'), 2))

In [ ]:
sns.heatmap(data=frame)

plt.show()